## AIM-5011-1 Natural Language Processing
## Assignment 1
## Zeyu Wang
## Fall 2021

## 1. Data load

I download old spam email tar data from https://spamassassin.apache.org/old/publiccorpus/ and use tarfile to extract MIME format data into two folde ham and spam. Then open them and load them into pandas.

In [15]:
from tarfile import open as open_tar

In [132]:
import tarfile
import re
import os
from email import message_from_file
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import defaultdict
from functools import partial
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score
)
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier

List all of the spam data.

In [17]:
ham_files_name_list = ['20021010_easy_ham.tar.bz2', '20021010_hard_ham.tar.bz2', '20030228_easy_ham.tar.bz2', '20030228_easy_ham_2.tar.bz2', '20030228_hard_ham.tar.bz2']
spam_files_name_list = ['20021010_spam.tar.bz2', '20030228_spam.tar.bz2', '20030228_spam_2.tar.bz2']

read ham data file into a list.

In [ ]:
ham_email_list = []
for ham_file in ham_files_name_list:
    file_extra_dir = './file/'+ 'ham/' +ham_file
    with open_tar('./file/'+ham_file) as tar:
        for tarinfo in tar:
            tar.extractall(file_extra_dir)
            if len(tarinfo.name.split('/')) > 1:
                with open(file_extra_dir+'/'+tarinfo.name, encoding='utf-8', errors='replace') as fp:
                    message = message_from_file(fp)
                    body = []
                    if message.is_multipart():
                        for b in message.get_payload():
                            body.append(str(b))
                        body = '\n'.join(body)
                    else:
                        body = message.get_payload()
                    ham_email_list.append(body)

read spam data into a list.

In [19]:
spam_email_list = []
for spam_file in spam_files_name_list:
    file_extra_dir = './file/'+ 'spam/' +spam_file
    with open_tar('./file/'+spam_file) as tar:
        for tarinfo in tar:
            tar.extractall(file_extra_dir)
            if len(tarinfo.name.split('/')) > 1:
                with open(file_extra_dir+'/'+tarinfo.name, encoding='utf-8', errors='replace') as fp:
                    message = message_from_file(fp)
                    body = []
                    if message.is_multipart():
                        for b in message.get_payload():
                            body.append(str(b))
                        body = '\n'.join(body)
                    else:
                        body = message.get_payload()
                    spam_email_list.append(body)

In [20]:
ham_pd = pd.DataFrame({'email':ham_email_list})

In [21]:
ham_pd['label'] = '0'#0 means ham

In [22]:
spam_pd = pd.DataFrame({'email':spam_email_list})
spam_pd['label'] = '1'#1 means spam

In [23]:
ham_pd

,email,label
0,\n> From: fork-admin@xent.com [mailto:fork-adm...,0
1,"Hi,\n\nOn Sun, 01 Sep 2002 00:05:03 MDT Reg Cl...",0
2,"On Fri Sep 13 2002 at 02:03, Robert Elz wrote:...",0
3,"On Thu, 2002-08-29 at 01:06, Matthias Saou wro...",0
4,"Hi, I'm building an rpm for the resin webserve...",0
...,...,...
6949,___ ___ ...,0
6950,IN THIS ISSUE:\n\n01. Readers write\n02. Exten...,0
6951,Content-Type: text/plain; charset=us-ascii\nCo...,0
6952,"Content-Type: text/plain;\n\tcharset=""ISO-8859...",0


In [24]:
spam_pd

,email,label
0,"Content-Type: text/plain;\n\tcharset=""iso-8859...",1
1,###################################\n\n FREE ...,1
2,"<table width=""600"" border=""20"" align=""center"" ...",1
3,<html>\n<head>\n<title>Untitled Document</titl...,1
4,Whiter teeth and a brighter smile are just a c...,1
...,...,...
2395,"Content-Type: text/plain;\n\tcharset=""Windows-...",1
2396,ATTN:SIR/MADAN \n\n ...,1
2397,"Dear Subscriber,\n\nIf I could show you a way ...",1
2398,****Mid-Summer Customer Appreciation SALE!****...,1


In [81]:
datalist = [ham_pd, spam_pd] 
data = pd.concat(datalist)
data

,email,label
0,\n> From: fork-admin@xent.com [mailto:fork-adm...,0
1,"Hi,\n\nOn Sun, 01 Sep 2002 00:05:03 MDT Reg Cl...",0
2,"On Fri Sep 13 2002 at 02:03, Robert Elz wrote:...",0
3,"On Thu, 2002-08-29 at 01:06, Matthias Saou wro...",0
4,"Hi, I'm building an rpm for the resin webserve...",0
...,...,...
2395,"Content-Type: text/plain;\n\tcharset=""Windows-...",1
2396,ATTN:SIR/MADAN \n\n ...,1
2397,"Dear Subscriber,\n\nIf I could show you a way ...",1
2398,****Mid-Summer Customer Appreciation SALE!****...,1


## 2. Data clean

When cleaned data, I delete some special characters and let them become lowercase.

In [353]:
data1 = data.copy()

In [354]:
def clean(data_email):
    '''
    This method is used to delete '\n', non-word, and number. Then lowercase the word.
    ------------------------------------------------------------
    This method will return the cleanning data
    '''
    data_email = re.sub(r'\n', ' ', data_email)
    data_email = re.sub(r'\W+', ' ', data_email)
    data_email = re.sub(r'[0-9]+', ' ', data_email)
    data_email = data_email.lower()
    return data_email

In [355]:
data1

,email,label
0,\n> From: fork-admin@xent.com [mailto:fork-adm...,0
1,"Hi,\n\nOn Sun, 01 Sep 2002 00:05:03 MDT Reg Cl...",0
2,"On Fri Sep 13 2002 at 02:03, Robert Elz wrote:...",0
3,"On Thu, 2002-08-29 at 01:06, Matthias Saou wro...",0
4,"Hi, I'm building an rpm for the resin webserve...",0
...,...,...
2395,"Content-Type: text/plain;\n\tcharset=""Windows-...",1
2396,ATTN:SIR/MADAN \n\n ...,1
2397,"Dear Subscriber,\n\nIf I could show you a way ...",1
2398,****Mid-Summer Customer Appreciation SALE!****...,1


In [86]:
data1['email'] = data1['email'].apply(lambda x: clean(x))

In [87]:
data1

,email,label
0,from fork admin xent com mailto fork admin xe...,0
1,hi on sun sep mdt reg clemens wrote ...,0
2,on fri sep at robert elz wrote date we...,0
3,on thu at matthias saou wrote thanks...,0
4,hi i m building an rpm for the resin webserver...,0
...,...,...
2395,content type text plain charset windows cont...,1
2396,attn sir madan strictly confidential i am plea...,1
2397,dear subscriber if i could show you a way to g...,1
2398,mid summer customer appreciation sale to expr...,1


## 3. method1 KNN

Here I will combine the NLP and KNN knowledge that I learn from the ML course. I think the KNN can do well in spam classification.

In [88]:
X = data1['email']
y = data1['label']

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [96]:
dictionary = defaultdict(int)

In [97]:
for email in X_train:
    for word in email.split(' '):
        dictionary[word] += 1

In [101]:
top = 1000
descending_dictionary = sorted(dictionary.items(), key=lambda v: v[1], reverse=True)
dictionary = [ word for (word, occur) in descending_dictionary if len(word) > 1][:top]

In [103]:
dictionary[0:10]

['the', 'font', 'to', 'td', 'http', 'com', 'and', 'of', 'width', 'br']

In [217]:
def encode_email(email,dictionary_, binary = False):
    encoded = np.zeros(len(dictionary_))
    words = email.split(' ')
    for word in words:
        index = np.where(dictionary_ == word)[0]
        if index.size == 1:
            if binary:
                encoded[index[0]] = 1
            else:
                encoded[index[0]] += 1
    return encoded

In [218]:
dictionary_np = np.array(dictionary)

In [219]:
X_train

1559    content type text plain charset iso     conten...
2778     doctype html public w c dtd html     transiti...
1713    url http jeremy zawodny com blog archives   ht...
3744    content type text plain charset us ascii from ...
1363    use perl daily newsletter in this issue two os...
                              ...                        
4671    url http scriptingnews userland com backissues...
4552    i ve been running hammie on all my incoming me...
555     url http www newsisfree com click       date n...
800     content type text html charset iso     content...
6881    reflexive entertainment s ion hardie gives us ...
Name: email, Length: 8418, dtype: object

In [250]:
encode_train = X_train.apply(lambda x: encode_email(x, dictionary_np))

In [224]:
encode_test = X_test.apply(lambda x: encode_email(x, dictionary_np))

In [258]:
encode_train1 = pd.DataFrame(encode_train)

In [259]:
encode_train

1559    [172.0, 198.0, 86.0, 101.0, 141.0, 178.0, 55.0...
2778    [43.0, 289.0, 43.0, 368.0, 159.0, 166.0, 25.0,...
1713    [2.0, 0.0, 0.0, 0.0, 1.0, 1.0, 3.0, 1.0, 0.0, ...
3744    [6.0, 0.0, 3.0, 0.0, 2.0, 4.0, 3.0, 0.0, 0.0, ...
1363    [2.0, 0.0, 4.0, 0.0, 9.0, 1.0, 6.0, 0.0, 0.0, ...
                              ...                        
4671    [2.0, 0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 0.0, ...
4552    [5.0, 0.0, 7.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, ...
555     [0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, ...
800     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
6881    [9.0, 0.0, 10.0, 0.0, 8.0, 8.0, 7.0, 5.0, 0.0,...
Name: email, Length: 8418, dtype: object

In [255]:
encode_train1

,email
441,"[10.0, 0.0, 7.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0,..."
4417,"[11.0, 0.0, 8.0, 0.0, 1.0, 1.0, 3.0, 3.0, 0.0,..."
4730,"[0.0, 0.0, 2.0, 0.0, 5.0, 4.0, 1.0, 0.0, 0.0, ..."
91,"[30.0, 0.0, 5.0, 0.0, 0.0, 0.0, 5.0, 18.0, 0.0..."
6636,"[9.0, 0.0, 13.0, 0.0, 0.0, 1.0, 1.0, 7.0, 0.0,..."
...,...
4654,"[1.0, 0.0, 0.0, 0.0, 4.0, 3.0, 0.0, 0.0, 0.0, ..."
5843,"[6.0, 0.0, 7.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, ..."
1444,"[23.0, 33.0, 14.0, 4.0, 3.0, 2.0, 18.0, 18.0, ..."
2288,"[9.0, 12.0, 5.0, 0.0, 3.0, 5.0, 3.0, 9.0, 0.0,..."


In [260]:
encode_train_split = pd.DataFrame(encode_train.to_list())

In [266]:
encode_test_split = pd.DataFrame(encode_test.to_list())

In [261]:
encode_train_split

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,172.0,198.0,86.0,101.0,141.0,178.0,55.0,39.0,52.0,81.0,...,0.0,0.0,0.0,0.0,3.0,6.0,0.0,0.0,0.0,0.0
1,43.0,289.0,43.0,368.0,159.0,166.0,25.0,28.0,228.0,100.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,1.0,1.0,3.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6.0,0.0,3.0,0.0,2.0,4.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,0.0,4.0,0.0,9.0,1.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8413,2.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8414,5.0,0.0,7.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8415,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [262]:
y_train

1559    0
2778    0
1713    0
3744    0
1363    0
       ..
4671    0
4552    0
555     0
800     1
6881    0
Name: label, Length: 8418, dtype: object

In [280]:
knn = KNeighborsClassifier()

In [281]:
knn.fit(encode_train_split, y_train)

KNeighborsClassifier()

In [286]:
label_predict = knn.predict(encode_test_split)

Calculate the accuracy of the test set.

In [287]:
len(label_predict==y_test)/len(label_predict)

1.0

In [272]:
labels_pred = cross_val_predict(knn,encode_train_split,y_train,cv=5)

In [277]:
print('accuracy:', accuracy_score(y_train, labels_pred))

accuracy: 0.9643620812544548


### 4. predict nornal in my email

Normal email:
Hi ZEYU WANG,

We’re making some changes to our legal agreements that will apply to you.

There is no action needed from you today, but if you would like to learn more, you can find details about these changes, when they apply and what you can do if you want to decline the changes on our Policy Updates page. You can also view these changes by visiting PayPal.com, clicking ‘Legal’ at the bottom of the page and then selecting ‘Policy Updates’.

If you have questions about any of these changes or your account, please don’t hesitate to get in touch with us.

Thank you for being a PayPal customer.

Sincerely,
PayPal

Spam email:
'Hello,Get help with all your homework questions big or small at  AceYourHomework.com . Our verified IVY League tutors are ready to help with any question you may have from basic math to complicated rocket science.GET HOMEWORK HELP, 30%OFF With Love,AceYourHomework Team'

In [339]:
nomal_email = ['Hi ZEYU WANG,We’re making some changes to our legal agreements that will apply to you.There is no action needed from you today, but if you would like to learn more, you can find details about these changes, when they apply and what you can do if you want to decline the changes on our Policy Updates page. You can also view these changes by visiting PayPal.com, clicking ‘Legal’ at the bottom of the page and then selecting ‘Policy Updates’.If you have questions about any of these changes or your account, please don’t hesitate to get in touch with us.Thank you for being a PayPal customer.SincerelyPayPal',
              'Hello,Get help with all your homework questions big or small at  AceYourHomework.com . Our verified IVY League tutors are ready to help with any question you may have from basic math to complicated rocket science.GET HOMEWORK HELP, 30%OFF With Love,AceYourHomework Team']

In [340]:
nornal_email_df = pd.DataFrame({'email':nomal_email})

In [341]:
nornal_email_df['labbel'] = [0,1]
nornal_email_df

,email,labbel
0,"Hi ZEYU WANG,We’re making some changes to our ...",0
1,"Hello,Get help with all your homework question...",1


In [342]:
nornal_email_df['email'] = nornal_email_df['email'].apply(lambda x: clean(x))

In [343]:
nornal_email_df

,email,labbel
0,hi zeyu wang we re making some changes to our ...,0
1,hello get help with all your homework question...,1


In [344]:
nornal_email_df = nornal_email_df['email'].apply(lambda x: encode_email(x, dictionary_np))

In [345]:
nornal_email_df

0    [3.0, 0.0, 5.0, 0.0, 0.0, 1.0, 2.0, 2.0, 0.0, ...
1    [0.0, 0.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...
Name: email, dtype: object

In [346]:
nornal_email_df = pd.DataFrame(nornal_email_df.to_list())

In [347]:
nornal_email_df

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,3.0,0.0,5.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [348]:
knn.predict(nornal_email_df)

array(['0', '1'], dtype=object)

### The predict is correct!

## Reference

https://medium.com/@thiagolcmelo/train-you-own-spam-detector-57725e8e81c0